In [1]:
import pandas as pd
import numpy as np

In [2]:
bureau = pd.read_csv('credit_risk/bureau.csv')

In [3]:
main = pd.read_csv('credit_risk/application_train.csv')
target = main[['TARGET', 'SK_ID_CURR']]
target.head()

,TARGET,SK_ID_CURR
0,1,100002
1,0,100003
2,0,100004
3,0,100006
4,0,100007


In [4]:
joined = bureau.merge(target, on='SK_ID_CURR', how='left')
joined.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,TARGET
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,0.0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,0.0
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,0.0
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,0.0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,0.0


In [8]:
import pandas as pd
import numpy as np
import os
import gc
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
# from xgboost import XGBClassifier as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE']
to_drop = []

In [49]:
# LABEL ENCODING
print("Encoding labels...")
for col in categorical_cols:
    enc = LabelEncoder()
    joined[col] = enc.fit_transform(joined[col])
    # self.labelEncoders[col] = enc

# DROP UNECESSARY FEATURES
print("Dropping features...")
for col in to_drop:
    joined.drop(col, inplace=True)

joined = replace_w_nan(joined)
joined.fillna(method='ffill', inplace=True)
# Creating dataset to store results in
df = pd.DataFrame()
df['SK_ID_CURR'] = joined['SK_ID_CURR'].unique()
df['TARGET'] = joined[['SK_ID_CURR', 'TARGET']].groupby('SK_ID_CURR').agg({'TARGET':'mean'})
df.dropna(subset=['TARGET'], inplace=True)


# FEATURE ENGINEERING
ops = ['min', 'max', 'mean', 'var']
# self.ops = ops

# JOINING WITH BUREAU_BALANCE
bureau_ids = joined[['SK_ID_BUREAU', 'SK_ID_CURR']]
bureau_balance = pd.read_csv('credit_risk/bureau_balance.csv')
bureau_balance = bureau_balance.merge(bureau_ids, on='SK_ID_BUREAU', how='inner')
bureau_balance = replace_w_nan(bureau_balance)
bureau_balance.fillna(method='ffill', inplace=True)

print("Encoding labels in bureau balance...")
enc = LabelEncoder()
bureau_balance['STATUS'] = enc.fit_transform(bureau_balance['STATUS'])
# self.labelEncoders['STATUS'] = enc


Encoding labels...
Dropping features...
Encoding labels in bureau balance...


In [50]:
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,SK_ID_CURR
0,5715448,0,6,380361
1,5715448,-1,6,380361
2,5715448,-2,6,380361
3,5715448,-3,6,380361
4,5715448,-4,6,380361


In [51]:
# Aggregates on bureau balance by bureau_id
balance_aggs = {
    'MONTHS_BALANCE':ops[:2],
    'STATUS':ops[:2] + [mode]
}
print("Applying aggregates on bureau balance ...")
bureau_balance_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(balance_aggs)
bureau_balance_agg.columns = pd.Index([e[0] + '_' + e[1].upper()  for e in bureau_balance_agg.columns.tolist()])
bureau_ids = bureau_balance_agg.merge(bureau_ids, on='SK_ID_BUREAU', how='right')
bureau_ids.fillna(method='bfill', inplace=True)
del bureau_balance_agg
gc.collect()

Applying aggregates on bureau balance ...


204

In [52]:
bureau_ids.head()

,SK_ID_BUREAU,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MAX,STATUS_MIN,STATUS_MAX,STATUS_CATEGORICAL_MODE,SK_ID_CURR
0,5001710,-82.0,0.0,0.0,7.0,6.0,162368
1,5001711,-3.0,0.0,0.0,7.0,0.0,162368
2,5001712,-18.0,0.0,0.0,6.0,0.0,162368
3,5001713,-21.0,0.0,7.0,7.0,7.0,150635
4,5001714,-14.0,0.0,7.0,7.0,7.0,150635


In [53]:
bureau_ids['STATUS_MIN'].value_counts()

0.0    662403
7.0     80209
6.0     30721
1.0       646
4.0       155
5.0       143
2.0        56
3.0        21
Name: STATUS_MIN, dtype: int64

1716428

In [ ]:
# Aggregating the aggregates by curr_id
nested_aggs = {
    # 'MONTHS_BALANCE_MIN': ops[:2],
    # 'MONTHS_BALANCE_MAX': ops[:2],
    'STATUS_MIN': ['min', mode],
    # 'STATUS_MAX': ['max', mode],
    # 'STATUS_CATEGORICAL_MODE': ops[:2]+[mode]
}
print("Applying aggregates on aggregated balance data...")
bureau_balance_agg = bureau_ids.groupby('SK_ID_CURR').agg(nested_aggs)
bureau_balance_agg.columns = pd.Index([e[0] + '_' + e[1].upper()  for e in bureau_balance_agg.columns.tolist()])
df = df.merge(bureau_balance_agg, on='SK_ID_CURR', how='left')
del bureau_balance_agg
gc.collect()


SK_ID_BUREAU               False
MONTHS_BALANCE_MIN          True
MONTHS_BALANCE_MAX          True
STATUS_MIN                  True
STATUS_MAX                  True
STATUS_CATEGORICAL_MODE     True
SK_ID_CURR                 False
dtype: bool

SK_ID_BUREAU               False
MONTHS_BALANCE_MIN         False
MONTHS_BALANCE_MAX         False
STATUS_MIN                 False
STATUS_MAX                 False
STATUS_CATEGORICAL_MODE    False
SK_ID_CURR                 False
dtype: bool